In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Data Visualization for TPL\n",
    "\n",
    "Visualize geometries, toolpaths, and fabrication data.\n",
    "\n",
    "**Author**: Zeyad Mustafa  \n",
    "**BTU Cottbus-Senftenberg**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from mpl_toolkits.mplot3d import Axes3D\n",
    "import seaborn as sns\n",
    "\n",
    "from tpl.design import Cube, Sphere, Cylinder, PathPlanner\n",
    "\n",
    "%matplotlib inline\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "sns.set_palette(\"husl\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. 3D Geometry Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create test geometries\n",
    "cube = Cube(size=10, center=(0, 0, 10))\n",
    "sphere = Sphere(radius=5, center=(15, 0, 10))\n",
    "cylinder = Cylinder(radius=3, height=15, center=(30, 0, 12.5))\n",
    "\n",
    "# Generate toolpaths\n",
    "planner = PathPlanner(layer_height=0.5, hatch_distance=0.5)\n",
    "\n",
    "tp_cube = planner.generate(cube)\n",
    "tp_sphere = planner.generate(sphere)\n",
    "tp_cylinder = planner.generate(cylinder)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 3D plot of all toolpaths\n",
    "fig = plt.figure(figsize=(15, 5))\n",
    "\n",
    "shapes = [('Cube', tp_cube), ('Sphere', tp_sphere), ('Cylinder', tp_cylinder)]\n",
    "\n",
    "for i, (name, tp) in enumerate(shapes, 1):\n",
    "    ax = fig.add_subplot(1, 3, i, projection='3d')\n",
    "    coords = tp.get_coordinates()\n",
    "    \n",
    "    ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], \n",
    "               c=coords[:, 2], cmap='viridis', s=0.5)\n",
    "    \n",
    "    ax.set_xlabel('X (μm)')\n",
    "    ax.set_ylabel('Y (μm)')\n",
    "    ax.set_zlabel('Z (μm)')\n",
    "    ax.set_title(f'{name} Toolpath')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Layer-by-Layer Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Separate toolpath by layers\n",
    "coords = tp_cube.get_coordinates()\n",
    "z_values = coords[:, 2]\n",
    "unique_layers = np.unique(z_values)\n",
    "\n",
    "print(f\"Number of layers: {len(unique_layers)}\")\n",
    "\n",
    "# Plot first 4 layers\n",
    "fig, axes = plt.subplots(2, 2, figsize=(12, 12))\n",
    "axes = axes.flatten()\n",
    "\n",
    "for i, z in enumerate(unique_layers[:4]):\n",
    "    layer_mask = (z_values >= z - 0.01) & (z_values <= z + 0.01)\n",
    "    layer_coords = coords[layer_mask]\n",
    "    \n",
    "    axes[i].plot(layer_coords[:, 0], layer_coords[:, 1], 'b-', linewidth=0.5)\n",
    "    axes[i].scatter(layer_coords[:, 0], layer_coords[:, 1], c='red', s=1)\n",
    "    axes[i].set_xlabel('X (μm)')\n",
    "    axes[i].set_ylabel('Y (μm)')\n",
    "    axes[i].set_title(f'Layer {i+1} (z={z:.2f} μm)')\n",
    "    axes[i].set_aspect('equal')\n",
    "    axes[i].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Dose Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate local doses\n",
    "doses = tp_cube.get_local_doses()\n",
    "\n",
    "fig, axes = plt.subplots(1, 2, figsize=(14, 5))\n",
    "\n",
    "# Histogram\n",
    "axes[0].hist(doses, bins=30, color='skyblue', edgecolor='black', alpha=0.7)\n",
    "axes[0].axvline(np.mean(doses), color='red', linestyle='--', \n",
    "                linewidth=2, label=f'Mean: {np.mean(doses):.2f}')\n",
    "axes[0].set_xlabel('Dose (a.u.)')\n",
    "axes[0].set_ylabel('Frequency')\n",
    "axes[0].set_title('Dose Distribution')\n",
    "axes[0].legend()\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Dose vs Z position\n",
    "axes[1].scatter(coords[:, 2], doses, alpha=0.3, s=1)\n",
    "axes[1].set_xlabel('Z Position (μm)')\n",
    "axes[1].set_ylabel('Dose (a.u.)')\n",
    "axes[1].set_title('Dose vs Height')\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"Dose Statistics:\")\n",
    "print(f\"  Mean: {np.mean(doses):.2f}\")\n",
    "print(f\"  Std Dev: {np.std(doses):.2f}\")\n",
    "print(f\"  Uniformity: {(1 - np.std(doses)/np.mean(doses))*100:.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Toolpath Statistics Dashboard"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "stats = tp_cube.get_statistics()\n",
    "\n",
    "fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n",
    "\n",
    "# Points per layer\n",
    "points_per_layer = len(coords) // stats['num_layers']\n",
    "layer_indices = np.arange(1, stats['num_layers'] + 1)\n",
    "axes[0, 0].bar(layer_indices[:20], [points_per_layer]*20, color='steelblue')\n",
    "axes[0, 0].set_xlabel('Layer Number')\n",
    "axes[0, 0].set_ylabel('Points per Layer')\n",
    "axes[0, 0].set_title('Points Distribution (First 20 Layers)')\n",
    "axes[0, 0].grid(True, alpha=0.3, axis='y')\n",
    "\n",
    "# Cumulative time\n",
    "cumulative_time = np.linspace(0, stats['time_estimate'], len(coords))\n",
    "axes[0, 1].plot(cumulative_time/60, coords[:, 2], linewidth=2)\n",
    "axes[0, 1].set_xlabel('Time (minutes)')\n",
    "axes[0, 1].set_ylabel('Z Position (μm)')\n",
    "axes[0, 1].set_title('Build Progress')\n",
    "axes[0, 1].grid(True, alpha=0.3)\n",
    "\n",
    "# Speed distribution\n",
    "axes[1, 0].hist(tp_cube.speeds, bins=20, color='coral', edgecolor='black')\n",
    "axes[1, 0].set_xlabel('Speed (μm/s)')\n",
    "axes[1, 0].set_ylabel('Frequency')\n",
    "axes[1, 0].set_title('Speed Distribution')\n",
    "axes[1, 0].grid(True, alpha=0.3)\n",
    "\n",
    "# Power distribution\n",
    "axes[1, 1].hist(tp_cube.powers, bins=20, color='lightgreen', edgecolor='black')\n",
    "axes[1, 1].set_xlabel('Power (mW)')\n",
    "axes[1, 1].set_ylabel('Frequency')\n",
    "axes[1, 1].set_title('Power Distribution')\n",
    "axes[1, 1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Comparative Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare different shapes\n",
    "import pandas as pd\n",
    "\n",
    "comparison = {\n",
    "    'Geometry': ['Cube', 'Sphere', 'Cylinder'],\n",
    "    'Volume (μm³)': [cube.get_volume(), sphere.get_volume(), cylinder.get_volume()],\n",
    "    'Points': [tp_cube.num_points, tp_sphere.num_points, tp_cylinder.num_points],\n",
    "    'Time (min)': [tp_cube.time_estimate/60, tp_sphere.time_estimate/60, tp_cylinder.time_estimate/60],\n",
    "    'Layers': [tp_cube.num_layers, tp_sphere.num_layers, tp_cylinder.num_layers]\n",
    "}\n",
    "\n",
    "df = pd.DataFrame(comparison)\n",
    "print(df)\n",
    "\n",
    "# Visualize comparison\n",
    "fig, axes = plt.subplots(1, 3, figsize=(15, 5))\n",
    "\n",
    "axes[0].bar(df['Geometry'], df['Volume (μm³)'], color=['blue', 'orange', 'green'])\n",
    "axes[0].set_ylabel('Volume (μm³)')\n",
    "axes[0].set_title('Geometry Volume')\n",
    "\n",
    "axes[1].bar(df['Geometry'], df['Points'], color=['blue', 'orange', 'green'])\n",
    "axes[1].set_ylabel('Number of Points')\n",
    "axes[1].set_title('Toolpath Complexity')\n",
    "\n",
    "axes[2].bar(df['Geometry'], df['Time (min)'], color=['blue', 'orange', 'green'])\n",
    "axes[2].set_ylabel('Time (minutes)')\n",
    "axes[2].set_title('Fabrication Time')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "This notebook demonstrated:\n",
    "\n",
    "1. ✓ 3D toolpath visualization\n",
    "2. ✓ Layer-by-layer inspection\n",
    "3. ✓ Dose distribution analysis\n",
    "4. ✓ Statistics dashboard\n",
    "5. ✓ Comparative geometry analysis\n",
    "\n",
    "Use these visualizations to:\n",
    "- Verify toolpath quality\n",
    "- Identify potential issues\n",
    "- Optimize parameters\n",
    "- Compare designs"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}